## Use the trapeziod method, simpson's rule, and romberg integration

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

Define a fxn for integration 

In [ ]:
def func(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return a*x**2 + b*x + c

Define its integral so we know the right answer

In [ ]:
def func_integral(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return (a*x**3)/3. + (b*x**2)/2. + c*x
#dviding by 3. means im dividing by a float 3

Define core of trapeziod method

In [ ]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x))
#f is fxn we want to integrate, x is the right side of interval
#h is width of interval

Define a wrapper(or driving) function to perform trapezoid method

In [ ]:
def trapezoid_method(f,a,b,N):
    #f==fxn to integrate
    #a==lower limit of integration
    #b==upper limit of integration
    #N== # of fxn evaluations to use
    #note # of chunks will be N-1
    #so if N is odd, then we dont need to adjust the last segment
    
    #define x values to perform trapeziod method
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define value of integral
    Fint = 0.0
    
    #perform integral using the trapeziod method, dont want to integrate past limits
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
    #f is fxn, x[i] is left hand side, h is width of interval
    #range starts from 0, len(x)-1 is the limit exclusive of iteration so its n-1
    #max value of 1 is n-2, need to stop at n-2
    
    #reutrn answer
    return Fint

Define core of simpson's method

In [ ]:
def simpson_core(f,x,h):
    return h*( f(x) + 4*f(x+h) + f(x+2*h))/3.

Define wrapper fxn to perform simpsons method

In [ ]:
def simpsons_method(f,a,b,N):
    #f==fxn to integrate
    #a==lower limit of integration
    #b==upper limit of integration
    #N== # of fxn evaluations to use
    #note # of chunks will be N-1
    #so if N is odd, then we dont need to adjust the last segment
    
    #define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define value of integral
    Fint = 0.0
    
    #if N is odd or even, we have different numbers of chunks to add
    if((N%2)==0):
        lim = 3
    else:
        lim = 2
    #the if statement= if N % 2 is even set it =3, if its odd=2
    #%=the remainder, if there isnt a remainder, then the fnx is even
    
    #perform integral using the simpsons method, dont want to integrate past limits
    for i in range(0,len(x)-2,2):
        Fint += simpson_core(f,x[i],h)
        print(i,i+2,x[i],x[i]+h,x[i]+2*h,x[-2],x[-1])
    #going from 0 to n-2 (n-3 inclusive) in steps of 2
    #2 is stride of interation
    #stopping at n-3, becasue were taking chunks 2 steps at a time
    #going to go up against n-1 depending on odd or even
    
    #in even case....
    #apply simpsons rule over the last interval if N is even
    if((N%2)==0):
        Fint += simpson_core(f,x[-2],0.5*h)
    #split the last interval in half to get 2 chunks
    
    #reutrn answer
    return Fint

Define the romberg integration core

In [ ]:
def romberg_core(f,a,b,I,i):
  # I is array of iterations getting successively better
    # i = current iteration factor, number of iterations
    
    #we need difference b-a, 1st width of 1st trapeziod
    h = b-a
    
    #and increment btwn new fxn evals, 2nd trapeziod
    dh = h/2.**(i)
    #every time we do a successively better integral, we cut our integral by factor of 2
    
    #we need the cofactor
    K = h/2.**(i+1)
    
    #and fxn evals, additional terms we need to evaluate in addition to 
    #integral evaluations i did in the previous step
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
    # far left side, plus half current width, adding succesivly terms of h
    #fxn evaluations, and adding by dh each time to the succesvie chunks
    
    #return the answer
    return K*M

Define wrapper fxn to perform romberg integration

In [ ]:
def romberg_integration(f,a,b,tol):
    
    #define integration variable
    i = 0
    
    #define max # of iterations
    imax = 1000
    
    #define error estimate, set to a large value, 1st error estimate
    delta = 100.0*np.fabs(tol)
    #we want answer within tolerance, we will iterate until we get our error
    #within the tolerance towards the end of the iteration
    
    #set array of integral answers
    I = np.zeros(imax,dtype=float)
    #array with imax # elements, store successively better answers
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while(delta>tol):
        #while loop
        
        #find this romberg iteration for when the error estimate is larger than allowed tol
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,I,i)
        
        #compute new fractional error estimate
        delta = np.fabs( (I[i]-I[i-1])/I[i] )
        #fabs=absolute value
        
        print(i,I[i],I[i-1],delta)
        
        if (delta>tol):
            
            #iterate
            i+=1
            
            #if weve reached the max iterations
            if(i>imax):
                print("max iterations reached.")
                raise StopIteration('Stopping iterations after ',i)
        
    #return the answer, if weve made it outside while loop
    return I[i]

In [ ]:
Answer = func_integral(1)-func_integral(0)
#integrating parabola between 0 and 1
print(Answer)
print("Trapezoid")
print(trapezoid_method(func,0,1,10))
print("Simpson Method")
print(simpsons_method(func,0,1,10))
print("Romberg")
tolerance = 1.0e-6
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)